# 🔗 Connect to Apache Cassandra & Settrade API (Fixed Version)
Notebook นี้ได้รับการแก้ไขให้สามารถเชื่อมต่อ Cassandra และ Settrade API ได้อย่างถูกต้องโดยไม่มี error

In [5]:
# 📦 Import Libraries
import pandas as pd
import yfinance as yf
from cassandra.cluster import Cluster
from datetime import datetime
import time


In [6]:
import yfinance as yf
import pandas as pd

# รายชื่อหุ้น
symbols = ["AOT.BK", 
           "PTT.BK",
           "CPALL.BK", 
           "ADVANC.BK"]

# ดึงข้อมูลพร้อมกัน
df = yf.download(symbols, start="2020-01-01", progress=False)

# ถ้าเป็นหลายหุ้น df จะเป็น multi-index columns
# รีเซ็ต index
df.reset_index(inplace=True)

# แปลง column names ให้เรียบร้อย
# สำหรับหลายหุ้น yfinance จะสร้าง column เป็น tuple เช่น ('Open', 'AOT.BK')
# เราสามารถ flatten columns
df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df.columns]
df.rename(columns={"Date_": "time"}, inplace=True)
df.dropna()
print(df)

print(df)



           time  Close_ADVANC.BK  Close_AOT.BK  Close_CPALL.BK  Close_PTT.BK  \
0    2020-01-02       172.591064     73.633636       64.390228     33.219006   
1    2020-01-03       171.007690     72.410484       65.070404     33.949085   
2    2020-01-06       171.799393     70.698082       64.616951     34.314133   
3    2020-01-07       176.549591     72.165855       65.523857     34.496655   
4    2020-01-08       174.174500     71.431976       65.750587     34.131611   
...         ...              ...           ...             ...           ...   
1398 2025-10-09       298.000000     43.000000       48.500000     32.000000   
1399 2025-10-10       295.000000     41.000000       48.000000     31.500000   
1400 2025-10-14       296.000000     39.750000       46.750000     31.000000   
1401 2025-10-15       298.000000     40.750000       47.250000     31.250000   
1402 2025-10-16       299.000000     40.500000       47.500000     30.750000   

      High_ADVANC.BK  High_AOT.BK  High

In [7]:
from cassandra.cluster import Cluster
from datetime import datetime

# ==========================================
# 1️⃣ เชื่อมต่อ Cassandra
# ==========================================
cluster = Cluster(['127.0.0.1'], port=9042)
session = cluster.connect()
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS data_stock
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};
""")
session.set_keyspace('data_stock')

session.execute("""
    CREATE TABLE IF NOT EXISTS candlestick_data (
        symbol text,
        time timestamp,
        open float,
        high float,
        low float,
        close float,
        volume bigint,
        value float,
        PRIMARY KEY (symbol, time)
    ) WITH CLUSTERING ORDER BY (time ASC);
""")
print("✅ Keyspace และ Table พร้อมใช้งาน!")

# ==========================================
# 2️⃣ ฟังก์ชันบันทึก DataFrame ลง Cassandra
# ==========================================
def insert_multiple_stocks(df, symbols):
    for symbol in symbols:
        print(f"\n📦 กำลังบันทึกข้อมูล {symbol} ...")
        # ตรวจ column ของหุ้นนั้น
        col_open = f"Open_{symbol}"
        col_high = f"High_{symbol}"
        col_low = f"Low_{symbol}"
        col_close = f"Close_{symbol}"
        col_volume = f"Volume_{symbol}"
        
        if col_open not in df.columns:
            print(f"⚠️ ไม่มีข้อมูลสำหรับ {symbol}")
            continue
        
        for idx, row in df.iterrows():
            timestamp = pd.Timestamp(row["time"]).to_pydatetime()
            session.execute("""
                INSERT INTO candlestick_data (
                    symbol, time, open_price, high_price, low_price,
                    close_price, volume, value
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                symbol,
                timestamp,
                float(row[col_open]),
                float(row[col_high]),
                float(row[col_low]),
                float(row[col_close]),
                int(row[col_volume]),
                float(row[col_close] * row[col_volume])
            ))
        print(f"✅ เพิ่มข้อมูล {len(df)} รายการของหุ้น {symbol} สำเร็จ!")


insert_multiple_stocks(df, symbols)


✅ Keyspace และ Table พร้อมใช้งาน!

📦 กำลังบันทึกข้อมูล AOT.BK ...
✅ เพิ่มข้อมูล 1403 รายการของหุ้น AOT.BK สำเร็จ!

📦 กำลังบันทึกข้อมูล PTT.BK ...
✅ เพิ่มข้อมูล 1403 รายการของหุ้น PTT.BK สำเร็จ!

📦 กำลังบันทึกข้อมูล CPALL.BK ...
✅ เพิ่มข้อมูล 1403 รายการของหุ้น CPALL.BK สำเร็จ!

📦 กำลังบันทึกข้อมูล ADVANC.BK ...
✅ เพิ่มข้อมูล 1403 รายการของหุ้น ADVANC.BK สำเร็จ!


In [8]:
# ==========================================
# 4️⃣ แสดงข้อมูลล่าสุดจาก Cassandra
# ==========================================
def show_latest_data(symbols, limit=5):
    for symbol in symbols:
        print(f"\n📊 ข้อมูลล่าสุดของ {symbol}:")
        
        # ดึง limit แถวล่าสุดเรียงตามเวลา
        rows = session.execute(f"""
            SELECT * FROM candlestick_data 
            WHERE symbol='{symbol}' 
            ORDER BY time DESC 
            LIMIT {limit}
        """)
        
        # แสดงผล
        for row in rows:
            print(f"เวลา: {row.time}, เปิด: {row.open_price}, สูง: {row.high_price}, ต่ำ: {row.low_price}, ปิด: {row.close_price}, ปริมาณ: {row.volume}, มูลค่า: {row.value}")

# เรียกใช้งานฟังก์ชัน
show_latest_data(symbols, limit=5)



📊 ข้อมูลล่าสุดของ AOT.BK:
เวลา: 2025-10-16 00:00:00, เปิด: 41.0, สูง: 41.5, ต่ำ: 40.0, ปิด: 40.5, ปริมาณ: 12869757, มูลค่า: 521225152.0
เวลา: 2025-10-15 00:00:00, เปิด: 40.75, สูง: 41.0, ต่ำ: 39.75, ปิด: 40.75, ปริมาณ: 19525100, มูลค่า: 795647808.0
เวลา: 2025-10-14 00:00:00, เปิด: 41.0, สูง: 41.0, ต่ำ: 39.25, ปิด: 39.75, ปริมาณ: 33248700, มูลค่า: 1321635840.0
เวลา: 2025-10-10 00:00:00, เปิด: 43.25, สูง: 43.25, ต่ำ: 40.75, ปิด: 41.0, ปริมาณ: 32693600, มูลค่า: 1340437632.0
เวลา: 2025-10-09 00:00:00, เปิด: 42.0, สูง: 43.0, ต่ำ: 41.75, ปิด: 43.0, ปริมาณ: 23732100, มูลค่า: 1020480320.0

📊 ข้อมูลล่าสุดของ PTT.BK:
เวลา: 2025-10-16 00:00:00, เปิด: 31.25, สูง: 31.25, ต่ำ: 30.5, ปิด: 30.75, ปริมาณ: 74223153, มูลค่า: 2282361856.0
เวลา: 2025-10-15 00:00:00, เปิด: 31.0, สูง: 31.25, ต่ำ: 31.0, ปิด: 31.25, ปริมาณ: 24356300, มูลค่า: 761134400.0
เวลา: 2025-10-14 00:00:00, เปิด: 31.5, สูง: 31.5, ต่ำ: 31.0, ปิด: 31.0, ปริมาณ: 69357200, มูลค่า: 2150073088.0
เวลา: 2025-10-10 00:00:00, เปิด: 32.0, สูง: 32.